In [1]:
!pip install catboost --quiet

     |████████████████████████████████| 64.8MB 64kB/s 


In [0]:
import pandas as pd 
import numpy as np 
import math
from tqdm import tqdm, tqdm_notebook
import gc
from collections import OrderedDict 
from sklearn.model_selection import KFold
from catboost import CatBoostRegressor
import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
sample_sub = pd.read_csv('sample_sub.csv')

In [5]:
train.head()

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target
0,ID_train_0,C,"nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...",45.126304
1,ID_train_1,D,"22.53333333,21.71666667,20.83333333,20.9833333...","0.102,0.0,0.0,0.0,0.0,0.0,0.0,0.034,0.017,0.01...","0.744583333,0.808083333,0.911166667,0.91633333...","281.6643101,89.15629262,81.96853891,291.018632...","2.3775,1.126666667,0.700833333,0.3416666670000...","90.32,90.3775,90.44083333,90.4725,90.45416667,...",79.131702
2,ID_train_10,A,"28.975,27.95,29.6,26.425,22.09166667,21.775,22...","0.0,0.0,0.0,0.102,0.136,0.0,0.0,2.16,1.276,0.0...","0.573333333,0.597166667,0.5668333329999999,0.6...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,na...","88.55166667,88.46416667,88.31916667,88.24,88.2...",32.661304
3,ID_train_100,A,"22.96666667,24.26666667,25.275,25.625,25.86666...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.77,3.012,1.0...","0.8430833329999999,0.79025,0.7375,0.728,0.7049...","300.0850574,293.6769595,294.5174647,301.921416...","1.446666667,1.1925,1.324166667,1.5441666669999...","88.615,88.53083333,88.4,88.27166667,88.2075,88...",53.850238
4,ID_train_1000,A,"21.875,21.575,21.525,21.43333333,20.50833333,1...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.8564166670000001,0.874916667,0.879833333,0.8...","21.83997432,17.05405341,89.26406044,123.585424...","0.1975,0.244166667,0.411666667,0.56,0.5775,0.4...","88.55666667,88.64083333,88.65833333,88.6475,88...",177.418750


In [6]:
test.head()

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press
0,ID_test_0,C,"23.96666667,22.88333333,22.28333333,21.4833333...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.6898333329999999,0.722333333,0.7755833329999...","290.1352379,328.47011289999995,323.5730796,282...","0.6333333329999999,0.725,0.955,0.8725,0.6925,0...","87.64333333,87.67,87.70583333,87.75083333,87.7..."
1,ID_test_1,C,"18.99166667,19.70833333,20.95833333,22.7833333...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.99975,0.99725,0.956333333,0.851583333,0.7775...","295.2350543,307.6569135,313.7360236,311.518385...","0.640833333,1.244166667,1.284166667,1.2875,1.3...","87.8925,87.95666667,87.9925,87.9925,87.9808333..."
2,ID_test_10,D,"25.11666667,25.08333333,24.40833333,23.575,22....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.760166667,0.733583333,0.78475,0.825833333,0....","77.06161145,95.48124997,61.44498258,89.4688360...","2.0133333330000003,1.3025,0.783333333,0.530833...","90.34,90.2975,90.305,90.35583333,90.4425,90.49..."
3,ID_test_100,C,"18.71666667,20.60833333,22.28333333,23.9833333...","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.604,0.0,0.0,...","0.951416667,0.92575,0.8603333329999999,0.77883...","310.7509058,316.186793,324.8167326,327.2174992...","0.745,1.169166667,1.316666667,1.385,1.40083333...","87.98833333,88.06333333,88.07833333,88.0366666..."
4,ID_test_1000,D,"17.63333333,18.05833333,19.625,21.28333333,23....","0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","0.9,0.898333333,0.8725,0.831666667,0.820833332...","334.6054069,329.6848561000001,322.7668036,312....","0.620833333,0.5,0.456666667,0.558333333,1.0158...","90.515,90.5575,90.62333333,90.6825,90.67916667..."


In [0]:
# covert features  fron string to List of values 
def replace_nan(x):
    if x==" ":
        return np.nan
    else :
        return float(x)
features=["temp","precip","rel_humidity","wind_dir","wind_spd","atmos_press"]
for feature in features : 
    train[feature]=train[feature].apply(lambda x: [ replace_nan(X) for X in x.replace("nan"," ").split(",")])
    test[feature]=test[feature].apply(lambda x: [ replace_nan(X)  for X in x.replace("nan"," ").split(",")])    

In [8]:
train.head()

,ID,location,temp,precip,rel_humidity,wind_dir,wind_spd,atmos_press,target
0,ID_train_0,C,"[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...",45.126304
1,ID_train_1,D,"[22.53333333, 21.71666667, 20.83333333, 20.983...","[0.102, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.034, 0...","[0.744583333, 0.808083333, 0.911166667, 0.9163...","[281.6643101, 89.15629262, 81.96853891, 291.01...","[2.3775, 1.126666667, 0.700833333, 0.341666667...","[90.32, 90.3775, 90.44083333, 90.4725, 90.4541...",79.131702
2,ID_train_10,A,"[28.975, 27.95, 29.6, 26.425, 22.09166667, 21....","[0.0, 0.0, 0.0, 0.102, 0.136, 0.0, 0.0, 2.16, ...","[0.573333333, 0.597166667, 0.5668333329999999,...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[nan, nan, nan, nan, nan, nan, nan, nan, nan, ...","[88.55166667, 88.46416667, 88.31916667, 88.24,...",32.661304
3,ID_train_100,A,"[22.96666667, 24.26666667, 25.275, 25.625, 25....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 7.77,...","[0.8430833329999999, 0.79025, 0.7375, 0.728, 0...","[300.0850574, 293.6769595, 294.5174647, 301.92...","[1.446666667, 1.1925, 1.324166667, 1.544166666...","[88.615, 88.53083333, 88.4, 88.27166667, 88.20...",53.850238
4,ID_train_1000,A,"[21.875, 21.575, 21.525, 21.43333333, 20.50833...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.8564166670000001, 0.874916667, 0.879833333,...","[21.83997432, 17.05405341, 89.26406044, 123.58...","[0.1975, 0.244166667, 0.411666667, 0.56, 0.577...","[88.55666667, 88.64083333, 88.65833333, 88.647...",177.418750


### Features engineering part

In [0]:
def aggregate_features(x,col_name):
    x["max_"+col_name]=x[col_name].apply(np.max)
    x["min_"+col_name]=x[col_name].apply(np.min)
    x["mean_"+col_name]=x[col_name].apply(np.mean)
    x["std_"+col_name]=x[col_name].apply(np.std)
    x["var_"+col_name]=x[col_name].apply(np.var)
    x["median_"+col_name]=x[col_name].apply(np.median)
    x["ptp_"+col_name]=x[col_name].apply(np.ptp)
    return x  
def remove_nan_values(x):
    return [e for e in x if not math.isnan(e)]


In [10]:
data=pd.concat([train,test],sort=False).reset_index(drop=True)
data.columns.tolist()

['ID',
 'location',
 'temp',
 'precip',
 'rel_humidity',
 'wind_dir',
 'wind_spd',
 'atmos_press',
 'target']

In [0]:
for x in range(121):
    data["temp_"+ str(x)] = data.temp.str[x]
    data["precip_"+ str(x)] = data.precip.str[x]
    data["rel_humidity_"+ str(x)] = data.rel_humidity.str[x]
    data["wind_dir_"+ str(x)] = data.wind_dir.str[x]
    data["wind_spd_"+ str(x)] = data.wind_spd.str[x]
    data["atmos_press_"+ str(x)] = data.atmos_press.str[x]    

In [12]:
data.shape

(20574, 735)

In [13]:
len(data.precip[10])

121

In [14]:
for col_name in tqdm(features):
    data[col_name]=data[col_name].apply(remove_nan_values)

100%|██████████| 6/6 [00:02<00:00,  2.54it/s]


In [15]:
for col_name in tqdm(features):
    data=aggregate_features(data,col_name)

100%|██████████| 6/6 [00:20<00:00,  3.43s/it]


In [0]:
data.drop(features,1,inplace=True)

In [0]:
train=data[data.target.notnull()].reset_index(drop=True)
test=data[data.target.isna()].reset_index(drop=True)

In [18]:
del data  
gc.collect()

103

In [0]:
# Concatenating train and test
train_len = train.shape[0]
df = pd.concat([train, test])

In [0]:
# Feature engineering, aggregating features by hour of day
for feat in features:
  for aggregate in ['kurt', 'skew']:
    df[aggregate+'_'+feat] = df[[x for x in df.columns if x.startswith(feat)]].agg(aggregate, axis = 1)

aggregates = ['mean', 'max',  'min', 'kurt', 'skew']
for feat in features:
  hours = OrderedDict()
  for i in range(24):
    hours['hour_' + str(i)] = []

  for i in range(24):
    for x, y in zip([int(j%24) for j in range(121)], [x for x in df.columns if x.startswith(feat)]):
      if x == i: hours.get('hour_' + str(i)).append(y)

  for i in range(24):
    for aggregate in aggregates:
      if aggregate in ['kurt', 'skew']:
        df[aggregate+'_'+feat+'_hour_'+str(i)] = df[hours.get('hour_' + str(i))].agg(aggregate)
      else:
        df[aggregate+'_'+feat+'_hour_'+str(i)] = df[hours.get('hour_' + str(i))].agg(aggregate, axis = 1)
# Binning values to speed up training and to remove noise
df[[x for x in df.columns if x not in ['ID', 'location', 'target']]] = \
df[[x for x in df.columns if x not in ['ID', 'location', 'target']]].\
apply(lambda x: pd.qcut(labels=False, duplicates='drop', q = 24, x = x))

In [0]:
# Splitting training and testing data from the concatenated dataframe
train = df[:train_len]
test = df[train_len:]

In [46]:
%%time
# Training model per location, per seed, while cross validating
seed_predictions = []
seed_predictions_2 = []
for i in tqdm_notebook(range(1)):
  loc_predictions = []
  loc_predictions_2 = []
  learning_rates = [0.052201, 0.03723, 0.043131, 0.051959, 0.047195]
  for location, rate in tqdm_notebook(zip(['A', 'B', 'C', 'D', 'E'], learning_rates), leave = False):
    X = train[(train.location == location)]
    y = X.target
    X = X.drop(['ID', 'location', 'target'], axis = 1)
    testt = test[(test.location == location)]
    testt = testt.drop(['ID', 'location', 'target'], axis = 1)
    if i in [0]:
      fold_predictions = []
      kfold = KFold(n_splits = 5, random_state = 0)
      for train_index, test_index in tqdm_notebook(kfold.split(X, y), leave = False):
        X.reset_index(drop = True, inplace=True), y.reset_index(drop = True, inplace = True)
        X_train, X_test, y_train, y_test = X.loc[train_index], X.loc[test_index], y[train_index], y[test_index]
        cat = CatBoostRegressor(verbose = False, eval_metric='RMSE', use_best_model=True, random_seed=i)
        cat.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], early_stopping_rounds=100)
        fold_predictions.append(cat.predict(testt))
      loc_predictions.extend(np.mean(fold_predictions, axis = 0))
    loc_predictions_2.extend(CatBoostRegressor(verbose = False, learning_rate=rate, random_seed=i).fit(X, y).predict(testt))
  seed_predictions.append(loc_predictions)
  seed_predictions_2.append(loc_predictions_2)
    
test_ids = []
for ids in ['A', 'B', 'C', 'D', 'E']:
  test_ids.extend(test[test.location == ids].ID)

mean_preds = ((np.mean(seed_predictions, axis = 0)*0.5) + (np.mean(seed_predictions_2, axis = 0)*0.5))
sub = pd.DataFrame({'ID': test_ids, 'target': mean_preds})
sub_1 = sample_sub.drop('target', axis = 1).merge(sub, how = 'left', on = 'ID')


CPU times: user 55min 35s, sys: 2min 51s, total: 58min 27s
Wall time: 31min 22s


In [47]:
%%time
# Training model on the full data
X = train.drop(['ID', 'location', 'target'], axis = 1)
y = train.target
testt = test.drop(['ID', 'location', 'target'], axis = 1)

predictions = []
kfold = KFold(n_splits = 20, random_state = 0)
for train_index, test_index in tqdm_notebook(kfold.split(X, y)):
  X_train, X_test, y_train, y_test = X.loc[train_index], X.loc[test_index], y[train_index], y[test_index]
  cat = CatBoostRegressor(verbose = False, iterations=20000, learning_rate=0.063602, eval_metric='RMSE', use_best_model=True)
  cat.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], early_stopping_rounds=100)
  predictions.append(cat.predict(testt))

sub = pd.DataFrame({'ID': test.ID, 'target': np.mean(predictions, axis = 0)})
sub_2 = sample_sub.drop('target', axis = 1).merge(sub, how = 'left', on = 'ID')


CPU times: user 4h 43min 59s, sys: 8min 2s, total: 4h 52min 1s
Wall time: 2h 33min 25s


In [0]:
# Ensembling the models and creating a submission file
ensemble = sub_1.target*0.5 + sub_2.target*0.5
pd.DataFrame({'ID': sub_1.ID, 'target': ensemble}).to_csv('unohana_final_sub.csv', index = False)